# - Data Warehouse -

### Het samenvoegen van bestaande tabellen voor het uiteindelijke 'Datawarehouse'

##### Importeren van benodigde dependencies

In [ ]:
import pandas as pd
import pyodbc

import warnings
warnings.filterwarnings("ignore")

# Importeren van de create_connection en run_query functies uit de database_utils.py file
from utils.database_utils import create_connection, run_query

##### 1 - Opbouwen van dataframes voor elke database-tabel

In [ ]:
aw_BusinessEntityAddressDF = run_query("SELECT * FROM Person.BusinessEntityAddress", "AdventureWorks2019")
aw_AddressDF = run_query("SELECT * FROM Person.Address", "AdventureWorks2019")
aw_AddressTypeDF = run_query("SELECT * FROM Person.AddressType", "AdventureWorks2019")
aw_StateProvinceDF = run_query("SELECT * FROM Person.StateProvince", "AdventureWorks2019")
aw_CountryRegionDF = run_query("SELECT * FROM Person.CountryRegion", "AdventureWorks2019")

print("BusinessEntityAddressDF columns:", aw_BusinessEntityAddressDF.columns)
print("AddressDF columns:", aw_AddressDF.columns)
print("AddressTypeDF columns:", aw_AddressTypeDF.columns)
print("StateProvinceDF columns:", aw_StateProvinceDF.columns)
print("CountryRegionDF columns:", aw_CountryRegionDF.columns)



Je kunt vergelijkbare queries uitvoeren voor andere tabellen die je wilt opnemen.

#### 2 - Data transformatie

Na het opbouwen van de dataframes voor elke database-tabel, kunnen we beginnen met het transformeren van de data. Dit omvat het samenvoegen van tabellen, het toepassen van filters, het uitvoeren van berekeningen, enzovoort.

In [ ]:
# Drop the 'rowguid' and 'ModifiedDate' columns from each DataFrame
aw_BusinessEntityAddressDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_AddressDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_AddressTypeDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_StateProvinceDF.drop(columns=['rowguid', 'ModifiedDate'], inplace=True)
aw_CountryRegionDF.drop(columns=['ModifiedDate'], inplace=True)

# Merge the DataFrames
merged_df = pd.merge(aw_BusinessEntityAddressDF, aw_AddressDF, on='AddressID', how='inner')
merged_df = pd.merge(merged_df, aw_AddressTypeDF, on='AddressTypeID', how='inner')
merged_df = pd.merge(merged_df, aw_StateProvinceDF, on='StateProvinceID', how='inner')
merged_df = pd.merge(merged_df, aw_CountryRegionDF, on='CountryRegionCode', how='inner')

filtered_df = merged_df.rename(columns={'Name': 'Country', 'Name_x': 'AddressType', 'Name_y': 'State'})
filtered_df = filtered_df.drop_duplicates(subset='BusinessEntityID', keep='first')
filtered_df

In [ ]:
create_table_query = """

CREATE TABLE AddressInfo (
    BusinessEntityID int PRIMARY KEY,
    AddressID int,
    AddressTypeID int,
    AddressLine1 nvarchar(max),
    AddressLine2 nvarchar(max),
    City nvarchar(max),
    StateProvinceID int,
    PostalCode nvarchar(max),
    SpatialLocation geography,
    AddressType nvarchar(max),
    StateProvinceCode nchar(10),
    CountryRegionCode nvarchar(max),
    IsOnlyStateProvinceFlag bit,
    State nvarchar(max),
    TerritoryID int,
    Country nvarchar(max),
);
"""

# Create the table in SQL Server
db_name = "testDB"
conn, cursor = create_connection(db_name)
cursor.execute(create_table_query)
conn.commit()
conn.close()

#### 3 - Data loading

Na het transformeren van de data, kunnen we de resulterende dataframe in de doeltabel van ons datawarehouse laden.

In [ ]:
test_database_name = 'testDB'
conn, cursor = create_connection(test_database_name)

for index, row in filtered_df.iterrows():
    # Convert binary data to hexadecimal
    spatial_location = row['SpatialLocation'].hex()

    # Convert Python boolean to SQL boolean
    is_only_state_province_flag = 1 if row['IsOnlyStateProvinceFlag'] else 0

    # Escape single quotes in string data
    address_line1 = row['AddressLine1'].replace("'", "''") if row['AddressLine1'] else None
    address_line2 = row['AddressLine2'].replace("'", "''") if row['AddressLine2'] else None
    city = row['City'].replace("'", "''") if row['City'] else None
    address_type = row['AddressType'].replace("'", "''") if row['AddressType'] else None
    state_province_code = row['StateProvinceCode'].replace("'", "''") if row['StateProvinceCode'] else None
    country_region_code = row['CountryRegionCode'].replace("'", "''") if row['CountryRegionCode'] else None
    state = row['State'].replace("'", "''") if row['State'] else None
    country = row['Country'].replace("'", "''") if row['Country'] else None

    query = f"""
        INSERT INTO AddressInfo  (
            BusinessEntityID, AddressID, AddressTypeID, AddressLine1, AddressLine2, City, 
            StateProvinceID, PostalCode, SpatialLocation, AddressType, 
            StateProvinceCode, CountryRegionCode, IsOnlyStateProvinceFlag, State, 
            TerritoryID, Country
        ) 
        VALUES (
            {row['BusinessEntityID']}, {row['AddressID']}, {row['AddressTypeID']}, '{address_line1}', 
            '{address_line2}', '{city}', {row['StateProvinceID']}, '{row['PostalCode']}', 
            0x{spatial_location}, '{address_type}', '{state_province_code}', 
            '{country_region_code}', {is_only_state_province_flag}, '{state}', 
            {row['TerritoryID']}, '{country}'
        )
    """
    print(query)  # Print the query
    cursor.execute(query)

conn.commit()
conn.close()

**Note:** Voeg indien nodig zoveel Markdown- of codeblokken toe als nodig is.

#### 4 -  Data Quality Checks

Voeg controles toe om de kwaliteit van de gegevens te waarborgen voordat ze worden geladen in het datawarehouse:

In [ ]:
# Controleren op ontbrekende waarden
missing_values = filtered_df.isnull().sum()

# Controleren op duplicaten
duplicate_rows = filtered_df.duplicated().sum()

# Weergave van resultaten
print("Aantal ontbrekende waarden:", missing_values)
print("Aantal duplicaten:", duplicate_rows)


**Note:** Dit is optioneel, het leek mij opzich best handig om te doen.